In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv('processed.csv')
print('Data shape:',df.shape)

Data shape: (1069, 22)


In [3]:
list(enumerate(df.columns))

[(0, 'w/b'),
 (1, 'Water'),
 (2, 'Cement type'),
 (3, 'Cement'),
 (4, 'Slag'),
 (5, 'Fly ash'),
 (6, 'Silica fume'),
 (7, 'Lime filler'),
 (8, 'FA'),
 (9, 'CA'),
 (10, 'Plasticizer'),
 (11, 'Superplasticizer'),
 (12, 'Air entraining'),
 (13, 'Comp. str. test age'),
 (14, 'Compressive strength'),
 (15, 'Air content'),
 (16, 'Spreed'),
 (17, 'Slump'),
 (18, 'Fresh density'),
 (19, 'Dry  density'),
 (20, 'Migration test age'),
 (21, 'Migration resistance')]

1-hot encoding

In [4]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

(1069, 30)
(1069,)


Grid search on RF

In [5]:
param_grid={
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': [10,50,100,200,300],
    'max_features': [None,1,2,3,4,5],
    'max_depth': [None,2,4,5,6],
    'min_samples_leaf': [1,3,4,5],
    'min_samples_split': [2,4,6,8,10],
    'bootstrap': [True,False]
}

splits=list(StratifiedKFold(shuffle=True,random_state=0).split(X,y))
train_index, test_index = splits[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_rdf(X_train,y_train):
    clf = RandomForestClassifier(random_state=0,n_jobs=-1)
    grid=GridSearchCV(clf,param_grid,cv=5,n_jobs=3,scoring='accuracy')
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

def cv_scores(X,y,model):
    acc_test=[]
    acc_train=[]
    for train_index, test_index in StratifiedKFold(shuffle=True,random_state=1).split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        model.fit(X_train,y_train)
        acc_test.append(accuracy_score(y_test,model.predict(X_test)))
        acc_train.append(accuracy_score(y_train,model.predict(X_train)))
    print('Train acc:',np.mean(acc_train))
    print('Test acc:',np.mean(acc_test))

In [6]:
grid=train_rdf(X_train,y_train)

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.8058479532163743


In [6]:
best_params={'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
best_model=RandomForestClassifier(random_state=0,**best_params)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97        43
         1.0       0.86      0.88      0.87        42
         2.0       0.78      0.74      0.76        43
         3.0       0.84      0.84      0.84        43
         4.0       0.95      0.95      0.95        43

    accuracy                           0.88       214
   macro avg       0.88      0.88      0.88       214
weighted avg       0.88      0.88      0.88       214



0.979516378154689

Removing cement type

In [7]:
X=df.drop('Cement type',axis=1).iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(1069, 20)
(1069,)


In [8]:
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
grid=train_rdf(X_train,y_train)

d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.8046783625730993


In [9]:
best_params={'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
best_model=RandomForestClassifier(random_state=0,**best_params)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

           0       0.89      0.95      0.92        43
           1       0.86      0.76      0.81        42
           2       0.73      0.74      0.74        43
           3       0.79      0.79      0.79        43
           4       0.93      0.95      0.94        43

    accuracy                           0.84       214
   macro avg       0.84      0.84      0.84       214
weighted avg       0.84      0.84      0.84       214



0.9746838025295798